In [73]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
sns.set()

In [74]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [75]:
train["title"] = train.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
test["title"] = test.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [76]:
train.Age.fillna(train.Age.median(), inplace=True)
train.Embarked.fillna('S', inplace=True)
del train['Cabin']
train.Sex.mask(train.Sex == 'male', 0, inplace=True)
train.Sex.mask(train.Sex == 'female', 1, inplace=True)
train.Embarked.mask(train.Embarked == 'S', 0, inplace=True)
train.Embarked.mask(train.Embarked == 'C', 1, inplace=True)
train.Embarked.mask(train.Embarked == 'Q', 2, inplace=True)

In [77]:
test.Age.fillna(train.Age.median(), inplace=True)
test.Embarked.fillna('S', inplace=True)
del test['Cabin']
test.Sex.mask(test.Sex == 'male', 0, inplace=True)
test.Sex.mask(test.Sex == 'female', 1, inplace=True)
test.Embarked.mask(test.Embarked == 'S', 0, inplace=True)
test.Embarked.mask(test.Embarked == 'C', 1, inplace=True)
test.Embarked.mask(test.Embarked == 'Q', 2, inplace=True)

In [166]:
x_train = pd.DataFrame([train.title, train.Sex, train.Embarked, train.Age, train.Pclass, train.Parch, train.SibSp, train.Survived]).T
x_test = pd.DataFrame([test.title, test.Sex, test.Embarked, test.Age, test.Pclass, test.Parch, test.SibSp]).T

In [167]:
x_train = x_train[x_train.title.isin(x_test.title.unique())]


In [168]:
title_dummies = pd.get_dummies(x_train.title, prefix='title').astype(int)
x_train = pd.concat([x_train, title_dummies], axis=1)
x_train = x_train.drop('title', axis=1)

In [169]:
y_train = x_train.Survived.astype(int)
x_train = x_train.drop('Survived', axis=1)
x_train.insert(loc=7, column='title_Dona', value=0)

In [170]:
title_dummies = pd.get_dummies(x_test.title, prefix='title').astype(int)
x_test = pd.concat([x_test, title_dummies], axis=1)
x_test = x_test.drop('title', axis=1)

In [171]:
x_train

,Sex,Embarked,Age,Pclass,Parch,SibSp,title_Col,title_Dona,title_Dr,title_Master,title_Miss,title_Mr,title_Mrs,title_Ms,title_Rev
0,0,0,22.0,3,0,1,0,0,0,0,0,1,0,0,0
1,1,1,38.0,1,0,1,0,0,0,0,0,0,1,0,0
2,1,0,26.0,3,0,0,0,0,0,0,1,0,0,0,0
3,1,0,35.0,1,0,1,0,0,0,0,0,0,1,0,0
4,0,0,35.0,3,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,27.0,2,0,0,0,0,0,0,0,0,0,0,1
887,1,0,19.0,1,0,0,0,0,0,0,1,0,0,0,0
888,1,0,28.0,3,2,1,0,0,0,0,1,0,0,0,0
889,0,1,26.0,1,0,0,0,0,0,0,0,1,0,0,0


In [172]:
x_test

,Sex,Embarked,Age,Pclass,Parch,SibSp,title_Col,title_Dona,title_Dr,title_Master,title_Miss,title_Mr,title_Mrs,title_Ms,title_Rev
0,0,2,34.5,3,0,0,0,0,0,0,0,1,0,0,0
1,1,0,47.0,3,0,1,0,0,0,0,0,0,1,0,0
2,0,2,62.0,2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,27.0,3,0,0,0,0,0,0,0,1,0,0,0
4,1,0,22.0,3,1,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,28.0,3,0,0,0,0,0,0,0,1,0,0,0
414,1,1,39.0,1,0,0,0,1,0,0,0,0,0,0,0
415,0,0,38.5,3,0,0,0,0,0,0,0,1,0,0,0
416,0,0,28.0,3,0,0,0,0,0,0,0,1,0,0,0


In [173]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(x_train, y_train)

c:\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [174]:
test['Survived'] = clf.predict(x_test)
submission = test[['PassengerId', 'Survived']]
submission.to_csv('submission.csv', index=False, sep=',')